In [2]:
import sys
assert sys.version_info >= (3, 5)
import time
import warnings
import os

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import sklearn.linear_model

import seaborn as sns

import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

from scipy.stats import pearsonr
warnings.filterwarnings("ignore")

df = pd.read_csv('country_vaccinations.csv')
df.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,Oxford/AstraZeneca,Government of Afghanistan,https://reliefweb.int/report/afghanistan/afgha...


In [10]:
df.shape
df.isna().sum()
train_df, test_df = train_test_split(df, shuffle=True, test_size = 0.95, random_state=17)
x = train_df['date']
x = [pd.to_datetime(d) for d in x]

y = train_df['people_vaccinated']
x2 = np.array(x).reshape(-1,1)
print(x2)
# model = sklearn.linear_model.LinearRegression()
# model.fit(x2,y)

# r_sq = model.score(x2, y)
# print(r_sq)

# plt.scatter(x2, y, color='black', label='observed')
# plt.plot(x2, model.predict(x2), label='fit', color='Green', linewidth=2)
# plt.xlabel('Date')
# plt.ylabel('People Vaccinated')
# plt.title('Regression')
# plt.legend(loc='best')

# plt.show()


[[Timestamp('2021-01-25 00:00:00')]
 [Timestamp('2021-03-19 00:00:00')]
 [Timestamp('2021-01-30 00:00:00')]
 [Timestamp('2020-12-28 00:00:00')]
 [Timestamp('2021-04-27 00:00:00')]
 [Timestamp('2021-04-08 00:00:00')]
 [Timestamp('2021-04-07 00:00:00')]
 [Timestamp('2021-02-24 00:00:00')]
 [Timestamp('2021-02-09 00:00:00')]
 [Timestamp('2021-04-17 00:00:00')]
 [Timestamp('2021-03-03 00:00:00')]
 [Timestamp('2021-03-16 00:00:00')]
 [Timestamp('2021-04-29 00:00:00')]
 [Timestamp('2021-04-07 00:00:00')]
 [Timestamp('2021-03-19 00:00:00')]
 [Timestamp('2021-02-09 00:00:00')]
 [Timestamp('2021-03-13 00:00:00')]
 [Timestamp('2021-01-14 00:00:00')]
 [Timestamp('2020-12-29 00:00:00')]
 [Timestamp('2021-02-18 00:00:00')]
 [Timestamp('2021-04-19 00:00:00')]
 [Timestamp('2021-04-05 00:00:00')]
 [Timestamp('2021-03-31 00:00:00')]
 [Timestamp('2021-03-30 00:00:00')]
 [Timestamp('2021-04-30 00:00:00')]
 [Timestamp('2021-02-10 00:00:00')]
 [Timestamp('2021-04-01 00:00:00')]
 [Timestamp('2021-03-10 00:0